In [1]:
# libs
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import mlflow

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Prepare Data

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
batch_sizes  = [32, 64, 128]
epochs       = [3,  5,  10]
dropouts     = [0.1, 0.25, 0.5]

## Modeling 

In [8]:
for batch_size, epoch_n, dropout in zip(batch_sizes, epochs, dropouts):
        mlflow.start_run()
        # architecture definition
        model = Sequential()
        model.add(Conv2D(filters=2, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
        model.add(Conv2D(filters=4, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dropout(dropout))
        model.add(Dense(num_classes, activation='softmax'))

        # compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

        # train model 
        model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epoch_n,
              verbose=1,
              validation_data=(x_test, y_test))

        # test model
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        # log model params and metrics
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("n_epochs", epoch_n)
        mlflow.log_metric("test_accuracy",  score[1])
        mlflow.log_metric("test_loss",      score[0])
        mlflow.end_run()

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 17s 283us/step - loss: 0.6352 - acc: 0.7860 - val_loss: 0.2927 - val_acc: 0.9107
Epoch 2/3
60000/60000 [==============================] - 18s 295us/step - loss: 0.4210 - acc: 0.8633 - val_loss: 0.2109 - val_acc: 0.9399
Epoch 3/3
60000/60000 [==============================] - 17s 290us/step - loss: 0.3498 - acc: 0.8852 - val_loss: 0.1760 - val_acc: 0.9473
Test loss: 0.17601574463546277
Test accuracy: 0.9473
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 168us/step - loss: 0.9753 - acc: 0.6526 - val_loss: 0.3451 - val_acc: 0.9139
Epoch 2/5
60000/60000 [==============================] - 10s 158us/step - loss: 0.6862 - acc: 0.7632 - val_loss: 0.2695 - val_acc: 0.9334
Epoch 3/5
60000/60000 [==============================] - 10s 160us/step - loss: 0.6105 - acc: 0.7904 - val_loss: 0.2210 - val_acc: 0.9461
Epoch 4/5
60000/600